In [1]:
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.8/802.8 kB 13.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.3 MB/s eta 0:00:00
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 48.8 MB/s eta 0:00:0000:010:01
  Created wheel for sagemaker: filename=sagemaker-2.164.0-py2.py3-none-any.whl size=1081792 sha256=a5a78b894e2326991987900934ad1ccb6728a5e317d42bf4e3ff3a3492691f5a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/28/5c/53/04f8b601a0cb36c00ddfb7ea379d82bc831163c6e0860eec17
Successfully built sagemaker
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.4.1
    Uninstallin

In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.estimator import Estimator
from sagemaker.session import s3_input

In [3]:
# SageMaker 실행 역할 가져오기
role = get_execution_role()

# SageMaker 세션 생성
sagemaker_session = sagemaker.Session()


In [20]:

# FastText 학습을 위한 데이터 경로
train_data = 's3://dothis-ai-test/data/video_data_20230414.txt'

# FastText 이미지 URI 가져오기
container = "118709199903.dkr.ecr.ap-northeast-2.amazonaws.com/fasttext_image:latest"

# 유효한 trainingJobName으로 변경
training_job_name = 'fasttext-job-1'

# FastText 추정기(Estimator) 생성
fasttext_estimator = Estimator(container, # FastText 모델을 실행하기 위한 Docker 컨테이너의 이미지 URI입니다.
                               role,
                               train_instance_count=1, # 학습 인스턴스의 수입니다. 병렬 학습을 위해 여러 인스턴스를 사용할 수 있습니다.
                               train_instance_type='ml.m5.xlarge', # 학습 인스턴스의 유형입니다. 인스턴스 유형은 학습 작업에 사용되는 리소스의 크기와 성능을 결정합니다.
                               train_volume_size=30, # 학습 인스턴스의 EBS 볼륨 크기입니다. 학습 데이터 및 모델 아티팩트를 저장하기 위해 필요한 디스크 공간을 지정합니다.
                               train_max_run=3600, # 학습 작업의 최대 실행 시간(초)입니다. 지정된 시간을 초과하면 학습 작업이 중지됩니다.
                               input_mode='File', # 입력 데이터의 형식을 지정하는 옵션입니다. 여기서는 "File"을 사용하여 파일 형식의 데이터를 지정합니다.
                               output_path='s3://dothis-ai-test/models', # 학습된 모델 아티팩트의 저장 위치입니다. 학습 작업이 완료된 후, 모델 아티팩트는 지정된 S3 경로에 저장됩니다.
                               sagemaker_session=sagemaker_session,
                               base_job_name=training_job_name)

# Hyperparameters 설정
fasttext_estimator.set_hyperparameters(bucket_size=2000000,
                                       dim=200,
                                       epoch=10,
                                       learning_rate=0.05,
                                       min_count=2,
                                       mode='cbow',
#                                        mode='skipgram',
                                       subwords=0,
                                       supervised=0,
                                       thread=4,
                                       verbose=0,
                                       word_ngrams=2)



See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [21]:
# 학습 데이터 구성
train_data_channel = s3_input(train_data, distribution='FullyReplicated')

# 학습 작업 실행
fasttext_estimator.fit(train_data_channel)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Creating training-job with name: fasttext-job-1-2023-06-09-08-56-25-200


2023-06-09 08:56:25 Starting - Starting the training job...
2023-06-09 08:56:43 Starting - Preparing the instances for training......
2023-06-09 08:57:37 Downloading - Downloading input data...
2023-06-09 08:58:24 Training - Training image download completed. Training in progress.
2023-06-09 08:58:24 Uploading - Uploading generated training model
2023-06-09 08:58:24 Failed - Training job failed
[FATAL tini (7)] exec train failed: No such file or directory


UnexpectedStatusException: Error for Training job fasttext-job-1-2023-06-09-08-56-25-200: Failed. Reason: AlgorithmError: , exit code: 127